In [1]:
from qutip import mesolve, sesolve, basis, tensor, destroy, Qobj, sigmax, sigmay, sigmaz, qeye
from qutip import liouvillian_ref, spre, spost, to_kraus
from sim_tools import id_wrap_ops, project_U, construct_basis_states_list
from qutip.qip.operations import hadamard_transform
from itertools import product
import numpy as np
import cmath

In [2]:
tmon_dim = 2
cavity_dim = 4
num_dof = 3
address_idx = 0
router_idx = 1
bus_idx = 2
truncated_dims = np.concatenate((list(product((cavity_dim,), repeat=2)), [tmon_dim]), axis=None)
hilbert_dim = np.prod(truncated_dims)
Fock_states_spec = [(i, 0, k) for i in range(2) for k in range(2)]

# below is for gf manifold stuff
# sz_gf = basis(3, 0) * basis(3, 0).dag() - basis(3, 2) * basis(3, 2).dag()
# sx_gf = basis(3, 0) * basis(3, 2).dag() + basis(3, 2) * basis(3, 0).dag()
# sy_gf = -1j * basis(3, 0) * basis(3, 2).dag() + 1j * basis(3, 2) * basis(3, 0).dag()
# had_gf = (1.0 / np.sqrt(2)) * (basis(3, 0) * basis(3, 0).dag() + basis(3, 0) * basis(3, 2).dag()
#                                + basis(3, 2) * basis(3, 0).dag() - basis(3, 2) * basis(3, 2).dag())
# Fock_states_spec = [(i, j, k) for i in range(2) for j in range(2) for k in [0, 2]]
#ee_Proj = id_wrap_ops(basis(3, 2) * basis(3, 2).dag(), bus_idx, truncated_dims)
# sz = id_wrap_ops(sz_gf, bus_idx, truncated_dims)
# sx = id_wrap_ops(sx_gf, bus_idx, truncated_dims)
# sy = id_wrap_ops(sy_gf, bus_idx, truncated_dims)
# had = id_wrap_ops(had_gf, bus_idx, truncated_dims)

# below is for ordering cavity, cavity, tmon
keep_idxs = [i * cavity_dim * tmon_dim + j * tmon_dim + k
             for (i, j, k) in Fock_states_spec]
ee_Proj = id_wrap_ops(basis(2, 1) * basis(2, 1).dag(), bus_idx, truncated_dims)
sz = id_wrap_ops(sigmaz(), bus_idx, truncated_dims)
sx = id_wrap_ops(sigmax(), bus_idx, truncated_dims)
sy = id_wrap_ops(sigmay(), bus_idx, truncated_dims)
had = id_wrap_ops(hadamard_transform(), bus_idx, truncated_dims)
# a -> address, r -> router dispersively coupled to qubit
r = id_wrap_ops(destroy(cavity_dim), router_idx, truncated_dims)
a = id_wrap_ops(destroy(cavity_dim), address_idx, truncated_dims)

In [3]:
def truncate_superoperator(superop, keep_idxs):
    """
    Parameters
    ----------
    superop
        superoperator to truncate. We consider the situation where certain
        states are relevant for predicting time evolution, however the gate under consideration
        does not care about the time evolution of those states themselves (with population 
        e.g. beginning in that state. 
    keep_idxs
        indices of the states to keep
    Returns
    -------
        truncated superoperator

    """
    keep_dim = len(keep_idxs)
    total_dim = hilbert_dim
    trunc_dim = total_dim - keep_dim
    truncated_mat = np.zeros((keep_dim ** 2, keep_dim ** 2), dtype=complex)
    total_dim = keep_dim + trunc_dim
    locs = [total_dim * keep_idx_i + keep_idx_j 
            for keep_idx_i in keep_idxs
            for keep_idx_j in keep_idxs]
    for i, loc_i in enumerate(locs):
        for j, loc_j in enumerate(locs):
            truncated_mat[i, j] = superop.data.toarray()[loc_i, loc_j]
    return Qobj(truncated_mat, type='super', dims=[[[keep_dim], [keep_dim]],
                                                   [[keep_dim], [keep_dim]]])

def my_to_chi(q_oper):
    """
    q_oper
        superoperator to transform into chi matrix
    """
    pauli_ops_oneq = [qeye(2) / 2, sigmax() / 2, 
                      sigmay() / 2, sigmaz() / 2]
    pauli_ops = [Qobj(tensor(pauli_op1, pauli_op2), dims=[[4], [4]]) for pauli_op1 in pauli_ops_oneq
                 for pauli_op2 in pauli_ops_oneq]
    kraus_ops = to_kraus(q_oper)
    e_ij_coeffs = np.array([[np.trace(kraus_op.dag() * pauli_op)
                             for pauli_op in pauli_ops]
                            for kraus_op in kraus_ops] 
                          )
    return np.conjugate(e_ij_coeffs).T @ e_ij_coeffs

def calc_fidel_chi(chi_real, chi_ideal):
    return (4 * np.trace(chi_real @ chi_ideal) + np.trace(chi_real))/5

In [64]:
chi = 2.0 * np.pi * 0.002 # 2 MHz chi
g = chi / 4
A = np.sqrt(2) * chi / 4
Hdbs = (g * a * r.dag() + np.conj(g) * a.dag() * r
     + 0.5 * chi * r.dag() * r * sz)
H0 = 0.5 * chi * sz * r.dag() * r
H0_e = -chi * ee_Proj * r.dag() * r
t_dbs = np.pi / (2 * A)
t_cz = t_idle = np.pi / chi

tmon_d_strength = 2.0 * np.pi * 0.01


def Rx(theta, c_ops=None):
    if c_ops is None:
        return (-1j * sx * theta / 2).expm()
    else:
        t = theta / tmon_d_strength
        return (liouvillian_ref(tmon_d_strength * sx / 2, c_ops) * t).expm()
    
    
def Ry(theta, c_ops=None):
    if c_ops is None:
        return (-1j * sy * theta / 2).expm()
    else:
        t = theta / tmon_d_strength
        return (liouvillian_ref(tmon_d_strength * sy / 2, c_ops) * t).expm()
    
    
def Rz(theta, c_ops=None):
    if c_ops is None:
        return (-1j * sz * theta / 2).expm()
    else:
        t = theta / tmon_d_strength
        return (liouvillian_ref(tmon_d_strength * sz / 2, c_ops) * t).expm()    
    
    
def U0(t, c_ops=None):
    if c_ops is None:
        return (-1j * t * H0).expm()
    else:
        return (liouvillian_ref(H0, c_ops) * t).expm()
    
    
def Udbs(t, c_ops=None):
    if c_ops is None:
        return (-1j * Hdbs * t).expm()
    else:
        return (liouvillian_ref(Hdbs, c_ops) * t).expm()
    
    
def U0_e(t, c_ops=None):
    if c_ops is None:
        return (-1j * t * H0_e).expm()
    else:
        return (liouvillian_ref(H0_e, c_ops) * t).expm()
    

def U_CNOT(c_ops=None):
    return Ry(-np.pi/2, c_ops=c_ops) * U0(t_cz, c_ops=c_ops) * Ry(np.pi/2, c_ops=c_ops)


def U_CNOT0(c_ops=None):
    return Rx(-np.pi, c_ops=c_ops) * U_CNOT(c_ops=c_ops)


def U_USWAP(c_ops=None):
    return (
        U0(t_idle, c_ops=c_ops) 
        * Rx(np.pi, c_ops=c_ops)
        * Udbs(t_dbs, c_ops=c_ops)
        * Rx(np.pi, c_ops=c_ops) 
        * Udbs(t_dbs, c_ops=c_ops) 
        * U0(t_idle, c_ops=c_ops)
    )

In [5]:
def construct_c_ops(Gamma_1_tmon, Gamma_phi_tmon, Gamma_1_res, Gamma_phi_res):
    c_ops = [np.sqrt(Gamma_1_tmon) * id_wrap_ops(basis(2, 0) * basis(2, 1).dag(), bus_idx, truncated_dims),
             np.sqrt(Gamma_phi_tmon) * sz,
             np.sqrt(Gamma_1_res) * a, 
             np.sqrt(Gamma_1_res) * r,
             np.sqrt(Gamma_phi_res) * a.dag() * a,  
             np.sqrt(Gamma_phi_res) * r.dag() * r,  
             ]
    return c_ops

In [10]:
# UCNOT = Ry(-np.pi / 2) * U0(t_cz) * Ry(np.pi / 2)
# #UCNOT = Ry(np.pi / 2) * U0_e(t_cz) * Ry(-np.pi / 2)
# #UCNOT0 = (-1)*Rx(-np.pi) * UCNOT
# UCNOT0 = Rx(-np.pi) * Ry(-np.pi / 2) * U0(t_cz) * Ry(np.pi / 2)
# U_USWAP = U0(t_idle) * Rx(np.pi) * Udbs(t_dbs) * Rx(np.pi) * Udbs * U0(t_idle)

In [67]:
def U_QRAM(D0, D1, c_ops=None):
    U_phase_computed = U0((D0 + D1) * t_cz, c_ops=c_ops)
    U_USWAP_computed = U_USWAP(c_ops=c_ops)
    if D0 == 0:
        if D1 == 0:
            return (
                U_USWAP_computed
                * U_phase_computed
                * U_USWAP_computed
            )
        else:
            return (
                U_USWAP_computed
                * U_phase_computed
#                * U0(t_cz, c_ops=c_ops)
                * U_CNOT(c_ops=c_ops)
                * U_USWAP_computed
            )
    else:
        if D1 == 0:
            return (
                U_USWAP_computed
                * U_phase_computed
#                * U0(t_cz, c_ops=c_ops)
                * U_CNOT0(c_ops=c_ops)
                * U_USWAP_computed
            )
        else:
            return (
                U_USWAP_computed 
                * U_phase_computed 
                * U_CNOT(c_ops=c_ops)
                * U_CNOT0(c_ops=c_ops)
#                * Rz(np.pi,c_ops=c_ops)
                * U_USWAP_computed
            )

In [ ]:
D0 = 0
D1 = 0

ideal_prop_truncated = project_U(U_QRAM(D0, D1), Fock_states_spec, truncated_dims)
chi_ideal = my_to_chi(spre(ideal_prop_truncated) * spost(ideal_prop_truncated.dag()))

In [7]:
T1_tmon = 150 * 10**3
Tphi_tmon = 200 * 10**3
T1_res = 1000 * 10**3
Tphi_res = 10000 * 10**3
c_ops = construct_c_ops(1/T1_tmon, 1/Tphi_tmon, 1/T1_res, 1/Tphi_res)

numer_prop = U_QRAM(D0, D1, c_ops=c_ops)
numer_prop_truncated = truncate_superoperator(numer_prop, keep_idxs)
chi_numer = my_to_chi(numer_prop_truncated)

calc_fidel_chi(chi_numer, chi_ideal)

(0.98151985932761+5.877471754111438e-40j)

In [46]:
QRAM_instance = U_QRAM(0, 1)
state00, state01, state10, state11  = construct_basis_states_list(((0, 0, 0), (0, 0, 1),
                                                                   (1, 0, 0), (1, 0, 1)), truncated_dims)

In [50]:
state11.dag() * QRAM_instance * state10

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.+1.j]]

In [70]:
U_QRAM_projected = project_U(U_QRAM(0, 1), Fock_states_spec, truncated_dims)
U_QRAM_projected * np.exp(-1j * cmath.phase(np.sum(U_QRAM_projected[0, :])))

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]]

In [288]:
U_USWAP_projected = project_U(U_USWAP, Fock_states_spec, truncated_dims)
U_USWAP_projected * np.exp(-1j * cmath.phase(np.sum(U_USWAP_projected[0, :])))

Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

In [25]:
U_CZ_projected = project_U(U0(3*t_idle), Fock_states_spec, truncated_dims)
U_CZ_projected * np.exp(-1j * cmath.phase(np.sum(U_CZ_projected[0, :])))

Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j]]

In [281]:
project_U(UCNOT0, Fock_states_spec, truncated_dims)

Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[ 0.+0.j  0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+1.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]]

In [14]:
project_U(UCNOT0, Fock_states_spec, truncated_dims)

Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

In [11]:
U_USWAP = U0(t_idle) * sx * Udbs * sx * Udbs * U0(t_idle)
project_U(U_USWAP, Fock_states_spec, truncated_dims)

Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

In [8]:
(tensor(basis(tmon_dim, 0), basis(cavity_dim, 0), basis(cavity_dim, 1)).dag() 
 * full_prop * tensor(basis(tmon_dim, 0), basis(cavity_dim, 1), basis(cavity_dim, 0))
)

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.+1.j]]

In [68]:
truncated_dims = [cavity_dim, cavity_dim]
a = id_wrap_ops(destroy(cavity_dim), 0, truncated_dims)
b = id_wrap_ops(destroy(cavity_dim), 1, truncated_dims)
Lx = a.dag() * b + a * b.dag()
Ly = -1j * (a.dag() * b - a * b.dag())
Lz = a.dag() * a - b.dag() * b
LI = a.dag() * a + b.dag() * b
Omega_I = 2.0 * np.pi * 0.67
Omega_x = 2.0 * np.pi * 0.43
Omega_y = 2.0 * np.pi * 0.3
Omega_z = 2.0 * np.pi * 0.23
varphi = np.arctan(Omega_y / Omega_x)
Omega_x_prime = np.sqrt(Omega_x**2 + Omega_y**2)
theta = np.arctan(Omega_x_prime / Omega_z)
Omega = np.sqrt(Omega_x**2 + Omega_y**2 + Omega_z**2)
t = 1.0
totalprop = (-1j * (Omega_x * Lx + Omega_y * Ly + Omega_z * Lz + Omega_I * LI) * t).expm()
prop1 = (-1j * (Omega_x * Lx + Omega_y * Ly + Omega_z * Lz)).expm()
prop2 = (-1j * varphi * Lz).expm() * (-1j * (Omega_x_prime * Lx)).expm() * (1j * varphi * Lz).expm()

In [69]:
Qobj((totalprop * a * totalprop.dag())[0:10,0:10])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j         -0.17857618+0.36136106j  0.        +0.j
   0.        +0.j          0.27762958+0.87203816j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.25254485+0.51104171j
   0.        +0.j          0.        +0.j          0.27762958+0.87203816j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.30930301+0.62589572j  0.        +0.j          0.        +0.j
   0.27762958+0.87203816j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j         -0.17127327+0.28535238j  0.        +0.j
   0.        +0.

In [79]:
np.exp(1j * Omega_I * t) * Qobj(((np.cos(Omega * t) + 1j * np.cos(theta) * np.sin(Omega * t)) * a 
 +1j * np.exp(-1j * varphi) * np.sin(Omega * t) * np.sin(theta) * b
)[0:10,0:10])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j         -0.17857618+0.36136106j  0.        +0.j
   0.        +0.j          0.27762958+0.87203816j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.25254485+0.51104171j
   0.        +0.j          0.        +0.j          0.27762958+0.87203816j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
  -0.30930301+0.62589572j  0.        +0.j          0.        +0.j
   0.27762958+0.87203816j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.27762958+0.87203816j  0.        +0.j
   0.        +0.

In [84]:
Qobj((totalprop * b * totalprop.dag())[1:11,1:11])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.83088554+0.99231381j  0.        +0.j
   0.        +0.j         -0.40079314-0.04285009j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.0176228 +1.21533124j
   0.        +0.j          0.        +0.j         -0.40079314-0.04285009j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.50994294+0.49180875j  0.        +0.j          0.        +0.j
   0.65344439-0.31141465j]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.5875248 +0.70167182j  0.        +0.j
   0.        +0.j         -0.56680709-0.06059917j  0.        +0.j
   0.        +0.

In [85]:
np.exp(1j * Omega_I * t) * Qobj(((np.cos(Omega * t) - 1j * np.cos(theta) * np.sin(Omega * t)) * b
 +1j * np.exp(1j * varphi) * np.sin(Omega * t) * np.sin(theta) * a
)[1:11,1:11])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.        +0.j          0.83088554+0.99231381j  0.        +0.j
   0.        +0.j         -0.40079314-0.04285009j  0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          1.0176228 +1.21533124j
   0.        +0.j          0.        +0.j         -0.40079314-0.04285009j
   0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.        +0.j          0.        +0.j
  -0.40079314-0.04285009j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j          0.5875248 +0.70167182j  0.        +0.j
   0.        +0.j         -0.56680709-0.06059917j  0.        +0.j
   0.        +0.

In [51]:
theta = 2.0 * np.pi * 0.3
prop3 = (1j * theta * Ly).expm() * a * (-1j * theta * Ly).expm()
prop4 = np.cos(theta) * a - np.sin(theta) * b

In [52]:
Qobj(prop3[0:10,0:10])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.         -0.95105652  0.          0.         -0.30901699  0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -1.34499702  0.          0.         -0.30901699
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.64727821  0.          0.
  -0.30901699  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         -0.003353    0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.95105652
   0.          0.         -0.43701602  0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -1.34499702  0.          0.         -0.43701602]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.04348496  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0

In [53]:
Qobj(prop4[0:10,0:10])

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.         -0.95105652  0.          0.         -0.30901699  0.
   0.          0.          0.          0.        ]
 [ 0.          0.         -1.34499702  0.          0.         -0.30901699
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         -1.64727821  0.          0.
  -0.30901699  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.         -0.30901699  0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.95105652
   0.          0.         -0.43701602  0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -1.34499702  0.          0.         -0.43701602]
 [ 0.          0.          0.          0.          0.          0.
   0.         -1.64727821  0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0

In [34]:
prop1 == prop2

True

In [35]:
prop1

Quantum object: dims = [[3, 4, 4], [3, 4, 4]], shape = (48, 48), type = oper, isherm = False
Qobj data =
[[ 1.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j -0.22425361+0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.05028968+0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 ...
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.00697846+0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.62765026+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  1.        +0.j]]

In [33]:
prop2

Quantum object: dims = [[3, 4, 4], [3, 4, 4]], shape = (48, 48), type = oper, isherm = False
Qobj data =
[[ 1.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j -0.22425361+0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.05028968+0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 ...
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.00697846+0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.62765026+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  1.        +0.j]]

In [47]:
# testing
Omega_x = 0.34
Omega_y = 0.83
Omega_z = 0.77
t = 10.0
varphi = np.arctan(Omega_y / Omega_x)
Omega_x_prime = np.sqrt(Omega_x**2 + Omega_y**2)
theta = np.arctan(Omega_x_prime / Omega_z)
Omega = np.sqrt(Omega_x**2 + Omega_y**2 + Omega_z**2)
prop1 = (-1j * (Omega_x * sigmax() + Omega_y * sigmay() + Omega_z * sigmaz()) * t).expm()
prop2 = ((-1j * 0.5 * varphi * sigmaz()).expm()
         * (-1j * 0.5 * theta * sigmay()).expm()
         * (-1j * Omega * sigmaz() * t).expm()
         * (1j * 0.5 * theta * sigmay()).expm()
         * (1j * 0.5 * varphi * sigmaz()).expm()
        )

In [48]:
prop1

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.73495015+0.44171041j  0.47612941+0.19504096j]
 [-0.47612941+0.19504096j  0.73495015-0.44171041j]]

In [49]:
prop2

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.73495015+0.44171041j  0.47612941+0.19504096j]
 [-0.47612941+0.19504096j  0.73495015-0.44171041j]]